In [2]:
import pandas as pd
import requests
import sqlite3
from pandas_gbq import to_gbq
import pandas_gbq
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta

C:\Users\REMO\AppData\Local\Temp\ipykernel_2480\4028859749.py:7 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [20]:
def download_data(**kwargs):
    url = "https://drive.google.com/uc?id=1nfktbI7ucHOUwO6EHD2pPHvOf1hN9nmZ"
    destination_file = "videogame.csv"
    response = requests.get(url)

    if response.status_code == 200:
        with open(destination_file, "wb") as file:
            file.write(response.content)
            print(f"The file {destination_file} has been successfully downloaded.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

    df = pd.read_csv(destination_file)
    return df


In [25]:
def transform_data(df):
    dataframes = {}
    dataframe_names = []

    df = df.dropna(axis=1, how='all')
    df = df.drop_duplicates()

    i = 1

    for columna in df.columns:
        if df[columna].isnull().any():
            df_temp = df.dropna(subset=[columna])
            df_temp = df_temp.dropna(axis=1, how='all')
            df_temp = df_temp.rename(columns={columna: 'Plataforma'})

            df_name = f'DF_{i}'
            dataframes[df_name] = df_temp
            dataframe_names.append(df_name)
            i += 1

    dataframes_list = []
    for df_name in dataframe_names:
        df_temp = dataframes[df_name]
        dataframes_list.append(df_temp)

    # Concatena los DataFrames de manera vertical
    df2 = pd.concat(dataframes_list, axis=0)


    df2[['Genero', 'Editorial']] = df2['Genero-Editorial'].str.rsplit('-', n=1, expand=True)
    df2 = df2.drop('Genero-Editorial', axis=1)

    df2['Ventas NA'] = df2['Ventas NA'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas EU'] = df2['Ventas EU'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas JP'] = df2[ 'Ventas JP'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas Otros'] = df2['Ventas Otros'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas Global'] = df2['Ventas Global'].str.replace(',', '.', regex=True).astype(float)
    return df2

In [18]:
def upload_data(df2):
    project_id = 'globanttd'
    dataset_id = 'Videogames'
    table_id = 'videogames1'

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = 'WRITE_TRUNCATE'  # Opcional: Sobrescribe la tabla existente.

    # Carga el DataFrame en BigQuery.
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job = client.load_table_from_dataframe(df2, table_ref, location='US', job_config=job_config)
    job.result()  # Espera a que se complete el trabajo de carga.

    print(f'Datos cargados en {project_id}.{dataset_id}.{table_id}')

In [28]:

default_args = {
    'owner': 'GORRO',
    'depends_on_past': False,
    'start_date': datetime(2023, 10, 12),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'my_data_pipeline',
    default_args=default_args,
    description='Un DAG para descargar, transformar y cargar datos',
    schedule=timedelta(minutes=10),  # Ejecutar cada 10 minutos
    catchup=False,
)

# Crea las tareas para descargar, transformar y cargar datos
download_task = PythonOperator(
    task_id='download_data',
    python_callable=download_data,
    dag=dag,
)

transform_task = PythonOperator(
    task_id='transform_data',
    python_callable=transform_data,
    dag=dag,
)

upload_task = PythonOperator(
    task_id='upload_data_to_bigquery',
    python_callable=upload_data,
    dag=dag,
)

# Define el orden de ejecución de las tareas
download_task >> transform_task >> upload_task

<Task(PythonOperator): upload_data_to_bigquery>

In [29]:
df2.describe()

NameError: name 'df2' is not defined